This notebook is a guide on how to operate the optimized nn-unet used in our paper (PAPER TITLE)
for more information you can visit the original repo of the segmentation network "https://github.com/NVIDIA/DeepLearningExamples/tree/master/PyTorch/Segmentation/nnUNet"

1-first step is to clone the repo of the network

In [ ]:
!git clone https://github.com/NVIDIA/DeepLearningExamples

Cloning into 'DeepLearningExamples'...
remote: Enumerating objects: 33148, done.
remote: Counting objects: 100% (971/971), done.
remote: Compressing objects: 100% (667/667), done.
remote: Total 33148 (delta 335), reused 858 (delta 279), pack-reused 32177
Receiving objects: 100% (33148/33148), 106.84 MiB | 25.61 MiB/s, done.
Resolving deltas: 100% (23249/23249), done.
Updating files: 100% (5338/5338), done.


now we will creat the folder in which we will put our data

In [ ]:
import os
os.makedirs("/content/data/BraTS2021_train")

Mounting our drive contatinig the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

unzipping the zipped data into our folder

The training dataset provided for the BraTS21 challenge consists of 1,251 brain mpMRI scans along with segmentation annotations of tumorous regions. The 3D volumes were skull-stripped and resampled to 1 mm isotropic resolution, with dimensions of (240, 240, 155) voxels. For each example, four modalities were given: Fluid Attenuated Inversion Recovery (FLAIR), native (T1), post-contrast T1-weighted (T1Gd), and T2-weighted (T2). See image below with each modality. Annotations consist of four classes: 1 for necrotic tumor core (NCR), 2 for peritumoral edematous tissue (ED), 4 for enhancing tumor (ET), and 0 for background (voxels that are not part of the tumor).

To download the training and validation dataset, you need to have an account on https://www.synapse.org platform and be registered for BraTS21 challenge. We will assume that after downloading and unzipping, the dataset is organized as follows:

```
/data 
 │
 ├───BraTS2021_train
 │      ├──BraTS2021_00000 
 │      │      └──BraTS2021_00000_flair.nii.gz
 │      │      └──BraTS2021_00000_t1.nii.gz
 │      │      └──BraTS2021_00000_t1ce.nii.gz
 │      │      └──BraTS2021_00000_t2.nii.gz
 │      │      └──BraTS2021_00000_seg.nii.gz
 │      ├──BraTS2021_00002
 │      │      └──BraTS2021_00002_flair.nii.gz
 │      ...    └──...
 │
 └────BraTS2021_val
        ├──BraTS2021_00001 
        │      └──BraTS2021_00001_flair.nii.gz
        │      └──BraTS2021_00001_t1.nii.gz
        │      └──BraTS2021_00001_t1ce.nii.gz
        │      └──BraTS2021_00001_t2.nii.gz
        ├──BraTS2021_00002
        │      └──BraTS2021_00002_flair.nii.gz
        ...    └──...
```

In [ ]:
!unzip "/content/drive/MyDrive/BraTS Validation Brain.zip" -d "/content/data/BraTS2021_train"

Installing the required libraries

In [ ]:
!pip install monai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 15.5 MB/s eta 0:00:00


In [ ]:
!pip install pytorch_lightning==1.9.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 827.8/827.8 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 729.2/729.2 kB 22.5 MB/s eta 0:00:00


In [ ]:
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-cuda110

Looking in indexes: https://pypi.org/simple, https://developer.download.nvidia.com/compute/redist
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.5/489.5 MB 1.9 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/NVIDIA/apex

Cloning into 'apex'...
remote: Enumerating objects: 11112, done.
remote: Counting objects: 100% (238/238), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 11112 (delta 131), reused 185 (delta 107), pack-reused 10874
Receiving objects: 100% (11112/11112), 15.39 MiB | 17.30 MiB/s, done.
Resolving deltas: 100% (7677/7677), done.


In [ ]:
import os
os.chdir("/content/apex")

'/content'

In [ ]:
!apt-get install ninja-build

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  ninja-build
0 upgraded, 1 newly installed, 0 to remove and 15 not upgraded.
Need to get 107 kB of archives.
After this operation, 338 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 ninja-build amd64 1.10.0-1build1 [107 kB]
Fetched 107 kB in 1s (111 kB/s)
Selecting previously unselected package ninja-build.
(Reading database ... 123105 files and directories currently installed.)
Preparing to unpack .../ninja-build_1.10.0-1build1_amd64.deb ...
Unpacking ninja-build (1.10.0-1build1) ...
Setting up ninja-build (1.10.0-1build1) ...
Processing triggers for man-db (2.9.1-1) ...


In [ ]:
!pip install -v --disable-pip-version-check --no-cache-dir --no-build-isolation --config-settings "--build-option=--cuda_ext" ./


In [ ]:
!pip install  'git+https://github.com/NVIDIA/dllogger'

  Cloning https://github.com/NVIDIA/dllogger to /tmp/pip-req-build-brhhbjgz
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/dllogger /tmp/pip-req-build-brhhbjgz
  Resolved https://github.com/NVIDIA/dllogger to commit 0540a43971f4a8a16693a9de9de73c1072020769
  Preparing metadata (setup.py) ... done
  Created wheel for DLLogger: filename=DLLogger-1.0.0-py3-none-any.whl size=5656 sha256=49a51dffa0959b604cc43d496d34b4a42076fc6bad62e2c2bb1be7fb5e8830e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-qk_e0eyb/wheels/41/52/02/4093763d9dcfd96d85b3d90dec4afdcf40b3934c0d41cd1f3c
Successfully built DLLogger


In [ ]:
!pip install -U rich

In [ ]:
os.chdir("/content")

Each example of the BraTS21 dataset consists of four NIfTI files with different MRI modalities (filenames with suffixes flair, t1, t1ce, t2). Additionally, examples in the training dataset have a NIfTI with annotation (filename with suffix seg). As a first step of data pre-processing, all four modalities are stacked such that each example has shape (4, 240, 240, 155) (input tensor is in the (C, H, W, D) layout, where C-channels, H-height, W-width and D-depth). Then redundant background voxels (with voxel value zero) on the borders of each volume are cropped, as they do not provide any useful information and can be ignored by the neural network. Subsequently, for each example, the mean and the standard deviation are computed within the non-zero region for each channel separately. All volumes are normalized by first subtracting the mean and then divided by the standard deviation. The background voxels are not normalized so that their value remained at zero. To distinguish between background voxels and normalized voxels which have values close to zero, we add an input channel with one-hot encoding for foreground voxels and stacked with the input data. As a result, each example has 5 channels.

Let's start by preparing the raw training and validation datasets into stacked NIfTI files.

We have two versions of this code when for when flair is Gans made and one for when the flair is original

The one below is for the Gans made

In [ ]:
import json
import os
from glob import glob
from subprocess import call
import time

import nibabel
import numpy as np
from joblib import Parallel, delayed
from skimage.exposure import rescale_intensity

def load_nifty(directory, example_id, suffix):
    return nibabel.load(os.path.join(directory, example_id + "_" + suffix + ".nii.gz"))


def load_channels(d, example_id):
    return [load_nifty(d, example_id, suffix) for suffix in ["flair", "t1", "t1ce", "t2"]]


def get_data(nifty, dtype="int16"):
    if dtype == "int16":
        data = np.abs(nifty.get_fdata().astype(np.int16))
        data[data == -32768] = 0
        return data
    return nifty.get_fdata().astype(np.uint8)

def get_data_flair(nifty, dtype="int16"):
    if dtype == "int16":
        data=nifty.get_fdata()
        data=rescale_intensity(data, out_range=(0, 1))
        data=data*32768
        data = np.abs(data.astype(np.int16))
        data[data == -32768] = 0
        return data
    return nifty.get_fdata().astype(np.uint8)

def prepare_nifty(d):
    example_id = d.split("/")[-1]
    flair, t1, t1ce, t2 = load_channels(d, example_id)
    affine, header = t1.affine, t1.header
    vol = np.stack([get_data_flair(flair), get_data(t1), get_data(t1ce), get_data(t2)], axis=-1)
    vol = nibabel.nifti1.Nifti1Image(vol, affine, header=header)
    nibabel.save(vol, os.path.join(d, example_id + ".nii.gz"))

    if os.path.exists(os.path.join(d, example_id + "_seg.nii.gz")):
        seg = load_nifty(d, example_id, "seg")
        affine, header = seg.affine, seg.header
        vol = get_data(seg, "unit8")
        vol[vol == 4] = 3
        seg = nibabel.nifti1.Nifti1Image(vol, affine, header=header)
        nibabel.save(seg, os.path.join(d, example_id + "_seg.nii.gz"))


def prepare_dirs(data, train):
    img_path, lbl_path = os.path.join(data, "images"), os.path.join(data, "labels")
    call(f"mkdir {img_path}", shell=True)
    if train:
        call(f"mkdir {lbl_path}", shell=True)
    dirs = glob(os.path.join(data, "BraTS*"))
    for d in dirs:
        if "_" in d.split("/")[-1]:
            files = glob(os.path.join(d, "*.nii.gz"))
            for f in files:
                if "flair" in f or "t1" in f or "t1ce" in f or "t2" in f:
                    continue
                if "_seg" in f:
                    call(f"mv {f} {lbl_path}", shell=True)
                else:
                    call(f"mv {f} {img_path}", shell=True)
        call(f"rm -rf {d}", shell=True)


def prepare_dataset_json(data, train):
    images, labels = glob(os.path.join(data, "images", "*")), glob(os.path.join(data, "labels", "*"))
    images = sorted([img.replace(data + "/", "") for img in images])
    labels = sorted([lbl.replace(data + "/", "") for lbl in labels])

    modality = {"0": "FLAIR", "1": "T1", "2": "T1CE", "3": "T2"}
    labels_dict = {"0": "background", "1": "edema", "2": "non-enhancing tumor", "3": "enhancing tumour"}
    if train:
        key = "training"
        data_pairs = [{"image": img, "label": lbl} for (img, lbl) in zip(images, labels)]
    else:
        key = "test"
        data_pairs = [{"image": img} for img in images]

    dataset = {
        "labels": labels_dict,
        "modality": modality,
        key: data_pairs,
    }

    with open(os.path.join(data, "dataset.json"), "w") as outfile:
        json.dump(dataset, outfile)


def run_parallel(func, args):
    return Parallel(n_jobs=os.cpu_count())(delayed(func)(arg) for arg in args)


def prepare_dataset(data, train):
    print(f"Preparing BraTS21 dataset from: {data}")
    start = time.time()
    run_parallel(prepare_nifty, sorted(glob(os.path.join(data, "BraTS*"))))
    prepare_dirs(data, train)
    prepare_dataset_json(data, train)
    end = time.time()
    print(f"Preparing time: {(end - start):.2f}")

prepare_dataset("/content/data/BraTS2021_train", True)
#prepare_dataset("/data/BraTS2021_val", False)
print("Finished!")

Preparing BraTS21 dataset from: /content/data/BraTS2021_train
Preparing time: 425.32
Finished!


you can notice in the above code there is a commented line, you can use this line when you want to preprocess the data to only predict not to train as it assumes that there is no ground truth labels

This one is for the original

In [ ]:
import json
import os
from glob import glob
from subprocess import call
import time

import nibabel
import numpy as np
from joblib import Parallel, delayed
from skimage.exposure import rescale_intensity

def load_nifty(directory, example_id, suffix):
    return nibabel.load(os.path.join(directory, example_id + "_" + suffix + ".nii.gz"))


def load_channels(d, example_id):
    return [load_nifty(d, example_id, suffix) for suffix in ["flair", "t1", "t1ce", "t2"]]


def get_data(nifty, dtype="int16"):
    if dtype == "int16":
        data = np.abs(nifty.get_fdata().astype(np.int16))
        data[data == -32768] = 0
        return data
    return nifty.get_fdata().astype(np.uint8)


def prepare_nifty(d):
    example_id = d.split("/")[-1]
    flair, t1, t1ce, t2 = load_channels(d, example_id)
    affine, header = t1.affine, t1.header
    vol = np.stack([get_data(flair), get_data(t1), get_data(t1ce), get_data(t2)], axis=-1)
    vol = nibabel.nifti1.Nifti1Image(vol, affine, header=header)
    nibabel.save(vol, os.path.join(d, example_id + ".nii.gz"))

    if os.path.exists(os.path.join(d, example_id + "_seg.nii.gz")):
        seg = load_nifty(d, example_id, "seg")
        affine, header = seg.affine, seg.header
        vol = get_data(seg, "unit8")
        vol[vol == 4] = 3
        seg = nibabel.nifti1.Nifti1Image(vol, affine, header=header)
        nibabel.save(seg, os.path.join(d, example_id + "_seg.nii.gz"))


def prepare_dirs(data, train):
    img_path, lbl_path = os.path.join(data, "images"), os.path.join(data, "labels")
    call(f"mkdir {img_path}", shell=True)
    if train:
        call(f"mkdir {lbl_path}", shell=True)
    dirs = glob(os.path.join(data, "BraTS*"))
    for d in dirs:
        if "_" in d.split("/")[-1]:
            files = glob(os.path.join(d, "*.nii.gz"))
            for f in files:
                if "flair" in f or "t1" in f or "t1ce" in f or "t2" in f:
                    continue
                if "_seg" in f:
                    call(f"mv {f} {lbl_path}", shell=True)
                else:
                    call(f"mv {f} {img_path}", shell=True)
        call(f"rm -rf {d}", shell=True)


def prepare_dataset_json(data, train):
    images, labels = glob(os.path.join(data, "images", "*")), glob(os.path.join(data, "labels", "*"))
    images = sorted([img.replace(data + "/", "") for img in images])
    labels = sorted([lbl.replace(data + "/", "") for lbl in labels])

    modality = {"0": "FLAIR", "1": "T1", "2": "T1CE", "3": "T2"}
    labels_dict = {"0": "background", "1": "edema", "2": "non-enhancing tumor", "3": "enhancing tumour"}
    if train:
        key = "training"
        data_pairs = [{"image": img, "label": lbl} for (img, lbl) in zip(images, labels)]
    else:
        key = "test"
        data_pairs = [{"image": img} for img in images]

    dataset = {
        "labels": labels_dict,
        "modality": modality,
        key: data_pairs,
    }

    with open(os.path.join(data, "dataset.json"), "w") as outfile:
        json.dump(dataset, outfile)


def run_parallel(func, args):
    return Parallel(n_jobs=os.cpu_count())(delayed(func)(arg) for arg in args)


def prepare_dataset(data, train):
    print(f"Preparing BraTS21 dataset from: {data}")
    start = time.time()
    run_parallel(prepare_nifty, sorted(glob(os.path.join(data, "BraTS*"))))
    prepare_dirs(data, train)
    prepare_dataset_json(data, train)
    end = time.time()
    print(f"Preparing time: {(end - start):.2f}")

prepare_dataset("/content/data/BraTS2021_train", True)
#prepare_dataset("/data/BraTS2021_val", False)
print("Finished!")

Preparing BraTS21 dataset from: /content/data/BraTS2021_train
Preparing time: 382.80
Finished!


Now, lets preprocesses the datasets by cropping, adding the one hot encoding and normalizing the volumes. We will store the pre-processed volumes as NumPy arrays.

In [ ]:
!python3 /content/DeepLearningExamples/PyTorch/Segmentation/nnUNet/preprocess.py --task 11  --exec_mode training --data "/content/data" --results "/content/data" --ohe
#!python3 ../preprocess.py --task 12 --ohe --exec_mode test
print("Finished!")

2023-07-13 16:23:58.668951: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 16:24:00.114150: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Preprocessing /content/data/BraTS2021_train
2023-07-13 16:24:16.784492: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-13 16:24:16.802472: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Pre-processing time: 637.68
Finished!


The line commented above is used when predicting not training

# training

the below command is used for training the model , you can know more about each argument in the nividia repo

In [ ]:
!python ../main.py --deep_supervision --depth 6 --filters 64 96 128 192 256 384 512 --min_fmap 2 --scheduler --learning_rate 0.0003 --epochs 30 --fold 0 --amp --gpus 1 --task 11 --save_ckpt

# Testing 

the below command is used for testing the model , you can know more about each argument in the nividia repo

In [ ]:
!python ../main.py --exec_mode "predict" --gpus 1 --depth 6 --filters 64 96 128 192 256 384 512 --min_fmap 2 --amp --save_preds --task 11 --data "/content/data/11_3d" --ckpt_path "/content/drive/MyDrive/epoch=7-dice=84.40.ckpt" --results "/content/drive/MyDrive/africa_60_nividia_results" --tta


Finally we will convert the .npy predections to the final predection form by taking the maximum propability 

In [ ]:
import os
from glob import glob
from subprocess import call

import nibabel as nib
import numpy as np
from scipy.ndimage.measurements import label


def to_lbl(pred):
    print(pred.shape)
    pred = np.argmax(pred, axis=0)

    """
    enh = pred[2]
    c1, c2, c3 = pred[0] > 0.5, pred[1] > 0.5, pred[2] > 0.5
    pred = (c1 > 0).astype(np.uint8)
    pred[(c2 == False) * (c1 == True)] = 2
    pred[(c3 == True) * (c1 == True)] = 4
    print(pred.shape)
    '''
    components, n = label(pred == 4)
    for et_idx in range(1, n + 1):
        _, counts = np.unique(pred[components == et_idx], return_counts=True)
        if 1 < counts[0] and counts[0] < 4 and np.mean(enh[components == et_idx]) < 0.9:
            pred[components == et_idx] = 1

    et = pred == 4
    if 0 < et.sum() and et.sum() < 5 and np.mean(enh[et]) < 0.9:
        pred[et] = 1
    '''
    """

    pred = np.transpose(pred, (2, 1, 0)).astype(np.uint8)
    print(pred.shape)
    return pred

def prepare_preditions(e):
    fname = e[0].split("/")[-1].split(".")[0]
    preds = [np.load(f) for f in e]
    p = to_lbl(np.mean(preds, 0))

    img = nib.load(f"/content/drive/MyDrive/finally_black_flair/data/BraTS2021_train/images/{fname}.nii.gz")
    nib.save(
        nib.Nifti1Image(p, img.affine, header=img.header),
        os.path.join("/content/drive/MyDrive/finally_black_flair/final_preds", fname + ".nii.gz"),
    )
!rm -r "/content/drive/MyDrive/finally_black_flair/final_preds"
os.makedirs("/content/drive/MyDrive/finally_black_flair/final_preds")
preds = sorted(glob(f"/content/drive/MyDrive/finally_black_flair/predictions_epoch=7-dice=84_40_task=11_fold=0_tta*"))
examples = list(zip(*[sorted(glob(f"{p}/*.npy")) for p in preds]))
print("Preparing final predictions")
for e in examples:
    prepare_preditions(e)
print("Finished!")